In [17]:
cuda_code = """
#include <iostream>
using namespace std;

__global__ void matrixMul(int *A, int *B, int *C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < n && col < n) {
        int value = 0;
        for (int k = 0; k < n; k++) {
            value += A[row * n + k] * B[k * n + col];
        }
        C[row * n + col] = value;
    }
}

int main() {
    int n = 3;
    int A[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    int B[] = {9, 8, 7, 6, 5, 4, 3, 2, 1};
    int C[n * n];

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, n * n * sizeof(int));
    cudaMalloc(&d_B, n * n * sizeof(int));
    cudaMalloc(&d_C, n * n * sizeof(int));

    cudaMemcpy(d_A, A, n * n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, n * n * sizeof(int), cudaMemcpyHostToDevice);

    dim3 blockSize(16, 16);
    dim3 gridSize((n + blockSize.x - 1) / blockSize.x, (n + blockSize.y - 1) / blockSize.y);
    matrixMul<<<gridSize, blockSize>>>(d_A, d_B, d_C, n);

    cudaDeviceSynchronize();

    cudaMemcpy(C, d_C, n * n * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "Result matrix C:\\n";
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            cout << C[i * n + j] << " ";
        }
        cout << endl;
    }

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    return 0;
}
"""

# Save CUDA code to a file
with open("matrix_mul.cu", "w") as f:
    f.write(cuda_code)

# Compile the CUDA code for the appropriate architecture (sm_70 for Tesla T4)
!nvcc -arch=sm_70 matrix_mul.cu -o matrix_mul

# Run the compiled program
!./matrix_mul

Result matrix C:
30 24 18 
84 69 54 
138 114 90 


In [ ]:
The program performs matrix multiplication (C = A × B) on two square matrices using the GPU via CUDA for parallel computation.

🚀 CUDA Kernel
cpp
Copy
Edit
__global__ void matrixMul(int *A, int *B, int *C, int n)
This is a CUDA kernel, a function that runs on the GPU.

The __global__ qualifier tells the compiler that this function can be called from the host (CPU) and executed on the device (GPU).

🧮 Matrix Multiplication Logic
Matrix multiplication is:

bash
Copy
Edit
C[i][j] = sum over k of A[i][k] * B[k][j]
Since matrices are stored in row-major order in C/C++, the element access is flattened:

cpp
Copy
Edit
A[row * n + k] * B[k * n + col]
Each GPU thread computes one value of the result matrix C[row][col].

💡 Important Keywords and Concepts Explained
Keyword	Explanation
__global__	CUDA specifier: kernel function run on GPU, called from host
blockIdx, threadIdx	Built-in CUDA variables: index of block/thread in grid
dim3	A CUDA struct for 3D block/grid dimensions (x, y, z)
cudaMalloc()	Allocates memory on GPU device
cudaMemcpy()	Transfers memory between host and device
cudaDeviceSynchronize()	Waits until all GPU threads are finished
cudaFree()	Frees memory allocated on GPU
nvcc	NVIDIA CUDA compiler

🧠 Broader Computer Science Concepts
Here are foundational CS concepts related to this code:

1. Parallel Computing
CUDA is all about data parallelism: the same operation (matrix multiply) is applied independently to different data (each matrix element).

Threads and blocks are the building blocks of parallel work on the GPU.

2. Memory Hierarchy
CUDA has multiple types of memory:

Global memory (cudaMalloc): accessible by all threads, but slower.

Shared memory: faster, used within thread blocks (not used in this code but crucial in optimization).

Registers/local memory: per-thread private memory.

Understanding memory hierarchy is key to optimizing performance.

3. Thread Mapping and Indexing
Each thread calculates its own output index:

cpp
Copy
Edit
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
This ensures that the grid of threads maps to the 2D output matrix.

4. Arithmetic Intensity and Computational Efficiency
Matrix multiplication has high arithmetic intensity (many computations per memory access), making it ideal for GPU acceleration.

GPUs outperform CPUs in such scenarios due to massive parallelism.

5. Synchronization and Race Conditions
cudaDeviceSynchronize() ensures all threads complete before copying results back.

No race condition in this example because each thread writes to its unique element in C.

🧾 Sample Output
rust
Copy
Edit
Result matrix C:
30 24 18
84 69 54
138 114 90
This confirms:

plaintext
Copy
Edit
A =  [1 2 3; 4 5 6; 7 8 9]
B =  [9 8 7; 6 5 4; 3 2 1]
C = A x B